In [1]:
# file_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\pdf-extractor\files"
# output_path =  r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\pdf-extractor\output"

path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"

samco_path = r"\output\TabularPdf.pdf"
tata_path = r"\files\TataFactSheet2024.pdf"

output_path = r"\output\TabulaTry.pdf"

In [74]:
import tabula
import pdfplumber
import fitz
import camelot
import warnings

warnings.filterwarnings("ignore", category=UserWarning) 

In [ ]:
tabula.environment_info()
file_path = path + samco_path
dfs = tabula.read_pdf(file_path, pages = "all", stream=True)

In [83]:
def extract_tables_by_page(file_path):
    """
    Extracts tables from each page of a PDF using Camelot and returns 
    a list of dictionaries, where each dictionary contains the page number 
    and a list of extracted tables for that page.

    Args:
        pdf_path: Path to the PDF file.

    Returns:
        list: A list of dictionaries, where each dictionary has 
              'pgnum' (page number) and 'tables' (list of extracted tables).
    """
    tables_by_page = []
    pdf = fitz.open(file_path)
    pdf_pages = pdf.page_count

    for pgn in range(1, pdf_pages + 1):
        try:
            page_tables = camelot.read_pdf(file_path, flavor='stream', pages=str(pgn))
        except Exception as e:
            #print(f"\nError extracting tables on page {pgn}: {e}")
            page_tables = []  # Handle errors gracefully

        bboxes = None
        tables = None
        blocks = None

        if page_tables:
            bboxes = [table._bbox for table in page_tables]
            tables = [table.df for table in page_tables]

        # Extract text blocks using fitz
        page = pdf[pgn - 1]  # Access page object (0-based indexing)
        blocks = page.get_text("blocks") 

        tables_by_page.append({
            "pgnum": pgn,
            "tables": tables,
            "bboxes": bboxes,
            "blocks": blocks,
        })

    return tables_by_page

    

In [84]:
page_tables = extract_tables_by_page(file_path)

In [89]:
len(page_tables)

tenth_page = page_tables[9]

for block in tenth_page['blocks']:
    print(f"\n {block}")




 (256.3013000488281, 134.12600708007812, 571.8199462890625, 148.93919372558594, 'Issuer\nIndustry\n% to Net\n', 0, 0)

 (546.3212890625, 142.28598022460938, 569.2742919921875, 151.9039764404297, 'Assets \n', 1, 0)

 (256.3013000488281, 166.62188720703125, 381.3462829589844, 707.2782592773438, 'Godawari Power And Ispat Ltd.\nSaregama India Limited\nEngineers India Limited\nApar Industries Limited\nJB Chemicals & Pharmaceuticals Limited\nPraj Industries Limited\nSuven Pharmaceuticals Limited\nAmara Raja Energy & Mobility Ltd\n360 One WAM Limited\nFinolex Cables Limited\nMotilal Oswal Financial Services Limited\nKSB Limited\nGujarat State Fertilizers & Chemicals Limited\nZensar Technologies Limited\nNational Aluminium Company Limited\nCastrol India Limited\neClerx Services Limited\nPCBL Limited\nGodfrey Phillips India Limited\nEmami Limited\nAjanta Pharma Limited\nCoromandel International Limited\nTorrent Pharmaceuticals Limited\nGlaxoSmithKline Pharmaceuticals Limited\nUnited Spirits Li

In [ ]:
table = tables[0]
type(table.df)

table.df.head(60)

In [17]:
table.parsing_report

{'accuracy': 99.97, 'whitespace': 23.19, 'order': 1, 'page': 4}